In [1]:
from pyspark.conf import SparkConf 
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession
from datetime import datetime
from dateutil.parser import parse
import pyspark.sql.functions as f
from pyspark.sql.types import *

import pyspark
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import findspark
findspark.find()
findspark.init(os.environ.get("$SPARK_HOME"))

## UserAgentParser
class UserAgentParser: 
    from user_agents import parse #// VERSION: 2.2.0
    from collections import namedtuple
    UA3Layers = namedtuple(typename='UA3Layers', field_names=['str_browser_name', 'str_operating_system_name', 'str_hardware_type_name'])
    
    class HardwareType:
        SERVER   = 'Server'
        TABLET   = 'Tablet'
        PHONE    = 'Phone'
        COMPUTER = 'Computer'
        OTHER    = 'Other'
        
    def get_3layers(self, str_user_agent):
        user_agent = self.__class__.parse(str_user_agent)
        return self.__class__.UA3Layers(self.get_browser_name(user_agent), self.get_operating_system_name(user_agent), self.get_hardware_type(user_agent))
    
    def get_browser_name(self, user_agent):
        """
        Return: ['Chrome', 'Firefox', 'Opera', 'IE', 'Edge', 'Safari', ...]
        변환로직:
            'IE Mobile' >>> 'IE'
            'Mobile Safari' >>> 'Safari'
        """
        str_browser_name = user_agent.browser.family
        str_browser_name = str_browser_name.replace('Mobile', '').strip()
        return str_browser_name
    
    def get_operating_system_name(self, user_agent):
        """
        Return: ['Windows','Linux','Mac OS X','iOS','Android','OpenBSD','BlackBerry OS','Chrome OS',...]
        """
        str_operating_system_name = user_agent.os.family
        return str_operating_system_name
    
    def get_hardware_type(self, user_agent):
        if user_agent.is_bot:
            str_hardware_type_name = self.__class__.HardwareType.SERVER
        elif user_agent.is_tablet:
            str_hardware_type_name = self.__class__.HardwareType.TABLET
        elif user_agent.is_mobile:
            str_hardware_type_name = self.__class__.HardwareType.PHONE
        elif user_agent.is_pc:
            str_hardware_type_name = self.__class__.HardwareType.COMPUTER
        else:
            str_hardware_type_name = self.__class__.HardwareType.OTHER
        return str_hardware_type_name


''' processing 단계: 
1. schema를 적용한 Dataframe 형태로 만들기

2. User Agents의 browser, OS, device 컬럼생성 및 기존컬럼제거

3. 발생할 수 있는 오류 확인 및 제거

4. row 제거 
- device를 기준으로 제거
- browser를 기준으로 제거
- os를 기준으로 제거

5. column 제거
- identifier 제거
- go-agent 제거
- protocol 제거

6. 변환
- 처리시간 변환
- 요청 시간을 datetime string으로 변환
- 형 변환
'''

' processing 단계: \n1. schema를 적용한 Dataframe 형태로 만들기\n\n2. User Agents의 browser, OS, device 컬럼생성 및 기존컬럼제거\n\n3. 발생할 수 있는 오류 확인 및 제거\n\n4. row 제거 \n- device를 기준으로 제거\n- browser를 기준으로 제거\n- os를 기준으로 제거\n\n5. column 제거\n- identifier 제거\n- go-agent 제거\n- protocol 제거\n\n6. 변환\n- 처리시간 변환\n- 요청 시간을 datetime string으로 변환\n- 형 변환\n'

In [2]:
# 1. schema를 이용한 dataframe 생성
columns=["IP","identifier","email","time","request","url","protocol","status_code",\
         "bytesize","referer","user-agent","go-agent","processing time"]

schema = StructType([ 
    StructField("IP", StringType(), True), 
    StructField("identifier", StringType(), True), 
    StructField("email", StringType(), True),
    StructField("time", StringType(), True),
    StructField("request", StringType(), True),
    StructField("url", StringType(), True),
    StructField("protocol", StringType(), True),
    StructField("status_code", StringType(), True),
    StructField("bytesize", StringType(), True),
    StructField("referer", StringType(), True),
    StructField("user-agent", StringType(), True),
    StructField("go-agent", StringType(), True),
    StructField("processing time", StringType(), True),
])

# 1. schema를 적용한 Dataframe 형태로 만들기
spark = pyspark.sql.SparkSession.builder.master('local[*]').getOrCreate()
web_log = spark.read.csv(f"hdfs://192.168.56.101:9000/web_log/log_01/access_2022-01-07.txt",  schema=schema)

## -------------------------------------------------------------------

AnalysisException: Path does not exist: hdfs://192.168.56.101:9000/web_log/log_01/access_2022-01-07.txt

In [3]:
# 2. User Agents의 browser, OS, device 컬럼생성 및 기존컬럼제거

def extract_browser(ua_string):
    userAgentParser = UserAgentParser()
    UA_info = userAgentParser.get_3layers(ua_string)
    return UA_info[0]

def extract_os(ua_string):
    userAgentParser = UserAgentParser()
    UA_info = userAgentParser.get_3layers(ua_string)
    return UA_info[1]

def extract_device(ua_string):
    userAgentParser = UserAgentParser()
    UA_info = userAgentParser.get_3layers(ua_string)
    return UA_info[2]

ext_browser_UDF = f.udf(lambda x: extract_browser(x))
ext_os_UDF = f.udf(lambda x: extract_os(x))
ext_device_UDF = f.udf(lambda x: extract_device(x))

web_log = web_log.withColumn("browser", ext_browser_UDF(f.col("user-agent")))
web_log = web_log.withColumn("os", ext_os_UDF(f.col("user-agent")))
web_log = web_log.withColumn("device", ext_device_UDF(f.col("user-agent")))
# web_log = web_log.drop("user-agent")

In [4]:
## -------------------------------------------------------------------

# 3. 발생할 수 있는 오류 확인 및 제거
web_log = web_log.filter((web_log.time  != "-") | (web_log.protocol  == "HTTP/1.0") | (web_log.protocol  == "HTTP/1.1"))

## -------------------------------------------------------------------

In [6]:
web_log.show()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 1]
Traceback (most recent call last):
  File "/usr/local/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt

KeyboardInterrupt



In [5]:
pd_log = web_log.toPandas()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 3]
Traceback (most recent call last):
  File "/usr/local/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt

KeyboardInterrupt



In [ ]:
len(pd_log) # 총 개수 

In [ ]:
# browser
plt.rcParams["figure.figsize"] = (5,7)
plt.title("Use Browser", fontsize=15)
a = pd_log["browser"].value_counts(ascending=True)
a[-20:].plot.barh(width = 0.5)

plt.ylabel("Browser", fontsize=12)
plt.xlabel("Count", fontsize=12)
plt.show()

In [77]:
pd_log1 = pd_log[pd_log["browser"] != "Electron"]

In [78]:
len(pd_log1)

1056238

In [ ]:
# os 
plt.rcParams["figure.figsize"] = (5,6)
plt.title("Use OS", fontsize=15)
pd_log["os"].value_counts(ascending=True).plot.barh(width = 0.5)

plt.ylabel("OS", fontsize=12)
plt.xlabel("Count", fontsize=12)
plt.show()

In [81]:
pd_log2 = pd_log1[pd_log1["os"] != "Other"]

In [82]:
len(pd_log2)

946983

In [ ]:
# device
plt.rcParams["figure.figsize"] = (5,6)
plt.title("Use Device", fontsize=15)
pd_log["device"].value_counts(ascending=True).plot.barh(width = 0.5)

plt.ylabel("Device", fontsize=12)
plt.xlabel("Count", fontsize=12)
plt.show()

In [ ]:
pd_log3 = pd_log2[pd_log2["device"] != "Other"]

In [ ]:
len(pd_log3)

In [84]:
set(pd_log2[pd_log2["device"] == "Other"]['user-agent'])

{'DaouOffice/10.3.0.20 (com.daou.GroupOfficeInhouse; build:20211118; iOS 14.4.2) Alamofire/5.4.3',
 'DaouOffice/10.3.0.20 (com.daou.GroupOfficeInhouse; build:20211118; iOS 15.1.0) Alamofire/5.4.3',
 'DaouOffice/10.3.0.20 (com.daou.GroupOfficeInhouse; build:20211118; iOS 15.1.1) Alamofire/5.4.3',
 'iOS/13.3 (17C54) dataaccessd/1.0',
 'iOS/14.6 (18F72) dataaccessd/1.0',
 'iOS/14.7 (18G68) dataaccessd/1.0',
 'iOS/14.7.1 (18G82) dataaccessd/1.0',
 'iOS/14.7.1 (18G82) remindd/1.0',
 'iOS/14.8 (18H17) dataaccessd/1.0',
 'iOS/14.8 (18H17) remindd/1.0',
 'iOS/14.8.1 (18H107) dataaccessd/1.0',
 'iOS/14.8.1 (18H107) remindd/1.0',
 'iOS/15.0.1 (19A348) dataaccessd/1.0',
 'iOS/15.0.2 (19A404) dataaccessd/1.0',
 'iOS/15.0.2 (19A404) remindd/1.0',
 'iOS/15.1 (19B74) dataaccessd/1.0',
 'iOS/15.1 (19B74) remindd/1.0',
 'iOS/15.1.1 (19B81) dataaccessd/1.0',
 'iOS/15.2 (19C56) dataaccessd/1.0',
 'iOS/15.2 (19C56) remindd/1.0',
 'iOS/15.2 (19C57) dataaccessd/1.0',
 'iOS/15.2 (19C57) remindd/1.0',
 'pytho

In [16]:
# 4. row 제거

# device : bot ,개발과정에서 발생한 로그(크롤링, curl,Zabbix, okhttp 등) 제거 
web_log = web_log.filter((web_log.device  != "Bot") | (web_log.device  != "Other"))

# browser : 개발과정에서 발생한 로그 제외
select_browser = ["Chrome",'Other', 'Whale', 'Safari UI/WKWebView', 'IE', 'Edge', 'Chrome WebView', 'Safari', 'Firefox', 'Outlook', 'Samsung Internet', 'Opera','Chrome iOS','Firefox iOS','Outlook-iOS']
web_log = web_log.filter(web_log.browser.isin(select_browser))

# os : 알 수 없는 os, 개발과정에서 발생한 로그 제외
web_log = web_log.filter((web_log.os != "Other"))

In [ ]:
# column 시각화

plt.rcParams["figure.figsize"] = (5,6)
plt.title("Use OS", fontsize=15)
pd_log3["identifier"].value_counts().plot(width = 0.5)

plt.ylabel("OS", fontsize=12)
plt.xlabel("Count", fontsize=12)
plt.show()

In [14]:
## -------------------------------------------------------------------

# 5. column 제거

# identifier : identifier count 0
web_log = web_log.drop("identifier")

# go-agent :  user-agent의 (browser, os, device)를 파악함으로써 go-agent의 정보를 대체할 수 있다.
web_log = web_log.drop("go-agent")

# protocol : http/1.0으로 중복, status_code를 통하여 http/1.0 protocol를 유추가능
web_log = web_log.drop("protocol")

## -------------------------------------------------------------------

# 6. 처리시간 변환
web_log = web_log.withColumn('processing time',\
                     f.when(web_log["processing time"].isin('-'),f.regexp_replace(web_log["processing time"],'-',''))\
                     .otherwise(web_log["processing time"]))

# 6. 요청 시간을 datetime string으로 변환
def trans_time(t):
    return str(pd.to_datetime(t,format='%d/%b/%Y:%H:%M:%S'))

time_UDF = f.udf(lambda x: trans_time(x))

web_log = web_log.withColumn("datetime", time_UDF(f.col("time")))
web_log = web_log.drop("time")


# 6. type 변환 (int, float, timestamp)
web_log = web_log.withColumn('bytesize',\
                     f.when(web_log["bytesize"].isin('-'),f.regexp_replace(web_log["bytesize"],'-','0')) .otherwise(web_log["bytesize"]))
web_log = web_log.withColumn("bytesize", web_log["bytesize"].cast(IntegerType()))                     

web_log = web_log.withColumn('processing time',\
                     f.when(web_log["processing time"].isin('-'),f.regexp_replace(web_log["processing time"],'-','0')) .otherwise(web_log["processing time"]))
web_log = web_log.withColumn("processing time", web_log["processing time"].cast(FloatType()))                

web_log = web_log.withColumn("datetime",f.to_timestamp("datetime"))

In [3]:
#             # PostgreSQL 적재 
#             web_log.write.mode("append").jdbc("jdbc:postgresql://localhost:5432/superset", f"public.admin{idx}",\
#                                           properties={"user": "postgres", "password": "1234"})

'''Pandas processing'''
# ''' processing : 
# 1. 읽은 파일을 데이터 프레임으로 변환
# 2. 컬럼 제거 / 제거기준 : "+0900", 클라이언트 식별자, 프로토콜 ,go-agent 식별정보
# 3. email -> userid를 뽑아내기
# 4. 요청 시간 -> datetime형태로 변환
# 5. UA -> browser, OS, device -> 기존컬럼제거
# 6  요청처리시간 오류 제거
# 7. int형태로 변환
# '''
    
# #  bite가 얼마나 줄었는지 생각하기 (size줄이기)
# #  postgreSQL에 적재할 테이블 기준생성
# #  ip/사용자아이디/요청 시간/요청 페이지/HTTP상태코드/바이트사이즈/레퍼러/요청처리시간/요청처리시간/브라우저/OS/장치 (13개컬럼)
# log = ['172.21.27.94 - cabeza_@daou.co.kr [17/Mar/2022:08:56:34 +0900] GET /api/chat/rooms/93730?size=21&userSeq=3012 HTTP/1.0 200 1757 "-" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) DaouMessenger/3.5.0 Chrome/91.0.4472.164 Electron/13.5.2 Safari/537.36" "GO-PC" 0.081',
# '112.168.170.128 - jieun.kim@daou.co.kr [17/Mar/2022:08:56:34 +0900] GET /api/approval/todo/count HTTP/1.0 200 224 "https://portal.daou.co.kr/app/approval/document/534993/integration" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36" "-" 0.129',
# '112.168.170.128 - jieun.kim@daou.co.kr [17/Mar/2022:08:56:34 +0900] GET /api/approval/document/534993 HTTP/1.0 200 6314 "https://portal.daou.co.kr/app/approval/document/534993/integration" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36" "-" 0.169',
# '123.2.134.129 - - [17/Mar/2022:08:57:02 +0900] GET /go/login HTTP/1.0 200 10162 "-" "Zabbix" "-" 0.015',
# '123.2.134.129 - - [17/Mar/2022:08:58:02 +0900] GET /go/login HTTP/1.0 200 10162 "-" "Zabbix" "-" 0.023',
# '123.2.134.129 - - [17/Mar/2022:08:59:04 +0900] GET /go/login HTTP/1.0 200 10162 "-" "Zabbix" "-" 0.017',
# '- - - [17/Mar/2022:08:59:29 +0900] GET / HTTP/1.1 302 - "-" "libwww-perl/5.822" "-" 0.019']

# columns = ["ip","user_id","datetime","request","url","status","bytesize","refferer","processing time", "browser", "OS", "device"]
# r= []
# def trans_email(t):
#     row[1] = t.split("@")[0]
    
# def extract_UAinfo(ua_string):
#     userAgentParser = UserAgentParser()
#     UA_info = userAgentParser.get_3layers(ua_string)
#     return [UA_info[0],UA_info[1],UA_info[2]]

# def trans_time(t):
#     print(type(str(pd.to_datetime(t,format='%d/%b/%Y:%H:%M:%S'))))
#     row[2] = pd.to_datetime(t,format='%d/%b/%Y:%H:%M:%S')

# for s in log:
#     s = s.replace('[', '"').replace(']','"')
#     s =s.split('"')

#     # 1. dataframe 만들기
#     row, tmp = [], []
#     for i in s:
#         if i==' ':
#             continue
#         i = i.strip()
#         tmp.append(i)

#     for i in tmp[:3]:
#         row += i.split()
#     row += tmp[3:]

#     # 2. 컬럼제거
#     del row[1]
#     del row[3]
#     del row[5]
#     del row[9]

#     # 3. email -> userid를 뽑아내기
#     trans_email(row[1])

#     # 4. 요청 시간 -> datetime형태로 변환
#     trans_time(row[2])

#     # 5. UA -> browser, OS, device -> 기존컬럼제거
#     row += extract_UAinfo(row[8])
#     del row[8]

#     # 6. 요청처리시간 오류 제거
#     row[8] = re.sub("-","",row[8])

#     # 7. int형태로 변환
#     if row[6] != '-':
#         row[6] = int(row[6])
#         row[8] = float(row[8])
        
#     r.append(row)


# df = pd.DataFrame(r, columns = columns)  
# df

s= '123425525asdf'
s[-4:-2]